# Introduction

The goal of forward kinematics is to find out where the robot's feet are from the joint angles we read from the robot. You'll do this by implementing a `forward_kinematics(q)` that takes in the joint angles (12-D) and return four 4x4 transformation matrices that represnt the pose of the feed frame in the robot's body frame (look at `02_fk_implementation.ipynb`). Since implementing this function might take you some time, in this notebook we ask you to record some dataset from the real robot so you can use them back home to validate your work. Next week you will usa your `forward_kinematics(q)` to do some fun experiments with the real robot! 


# Dataset Recording 

## Robot Bridge

If the robot bridge from the previous notebook is still running you can skip to the next step. Otherwise, follow the instructions in `00_interface_and_sim.ipynb` ("Starting the Bridge" section). 

## Connect to the Robot

Like the previous notebook, you'll need to import the `RobotReal` and instantiate it:

In [ ]:
from rob2004.interface import RobotReal
robot_real = RobotReal(port=4243)

Make sure you can read the robot states before continuing. Move the robot joints and run the next cell multiple times to make sure the values change as you move the legs:

In [ ]:
state = robot_real.getJointState()
print(state['q'])

{'q': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'dq': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'stamp': array(1.77051415e+09)}


## Recording Datasets

Here we ask you to record two sanity checking datasets and one as you desire.

- **Linear Up-Down Motion**: For this dataset, you must lean the front foot against a vertical surface and gradually move the foot up and down along the world z-axis. You must make sure you minimize any non-vertical movements
- **Touching Two Feet**: For this dataset, you start from a distance and gradually bring together the front right and back right feet unitl they touch. You repeat this multiple time such that the in one sequence, the feet make contact two or three times. 
- **You decide!(optional)**: You can come up with your own motion and record a dataset as you desire!

Use the next cell to record the dataset (a list of joint angles every 100ms). You must record a video while collecting the data and include it in your report. 

In [ ]:
import time
joint_angles = []
start_time = time.time()
while time.time() < 60:
    state = robot_real.getJointState()
    q = state['q'].copy().tolist()
    joint_angles.append(q)

Make sure you save each dataset with its own distinctive name at the end of each recording:

In [ ]:
import pickle

dataset_file_name = "*.pkl" # change the name for each dataset

with open(dataset_file_name, 'wb') as f:
    pickle.dump(joint_angles)

## Simulation Playback
To make sure your data is recorded correctly, here you must use the simulator to playback the joint angles you saved in each dataset in the simulator. First run the simulator app in visualization model as explaned in the previous notebook and then connect to it here:

In [ ]:
from rob2004.interface import RobotSim
robot_sim = RobotSim(port=4242)

**Your Task**: Complete the `play_dataset` function and run it on all the datasets you recorded. You must share the screen recording of this playback and upload it alongside the video you recorded with your phone while collecting each dataset.

In [ ]:
def play_dataset(dataset_file, robot_sim):
    with open(dataset_file, 'rb') as f:
        joint_angles = pickle.load(f)
    
    # Todo: loop over the joint_anbles and send them to the simulator. Note that you must use time.sleep() with the same wait time as recording loop.
    # Hint: for q in joint_angles:  


# Real Robot Bridge

## Starting the Bridge 
Now that you have the simulator, let's see how we can control the real robot! If you remember, in the previous lab you had to launch several ros nodes. The procedure here is very similar. First SSH into the robot:

```bash
ssh pi@robot_ip # Get the IP address of the robot the same way you did in Lab1
```

When logged in, navigate to the rob2004 repository on the robot:

```bash 
cd ~/rob2004
ros2 launch scripts/robot.launch.py
```
with running above, the robot's joints should initialize like what you saw in the previous lab. Now open anohter terminal and SSH into the robot again:
```bash
ssh pi@robot_ip 
rob2004-robot-bridge --port 4243
```
If successful, the robot is now ready and waiting for you to connect to it with the `RobotReal` class!

## Connecting to the Robot
We interact with the robot through a Python class with the exact same interface as the simulated robot. This way you'll be able to run your algorithms in simulation and easity transfer to the real robot when you come to the lab! Import the interface class and instantiate it. You'll need the IP address of your robot (the same address you used to SSH into it) for this:

In [ ]:
from rob2004.interface import RobotReal

robot_ip="" #Enter your robot's IP here
robot_real = RobotReal(ip=robot_ip, port=4243)

If everything is done right, you should be able to read the robot's joint states exactly like before:

In [ ]:
state = robot_sim.getJointState()
print(state)

**Your Task**: Run the commands above repeatedly in a loop and see the joint values (`q`) changing as you move the robot's joints.

In [2]:
import time  # Important: use time.sleep(0.1) to wait for 100 ms in between each update
# ToDo: Write a loop to print the joint positions q for 100 times

# Mirroring the Real Robot Movement In Simulation

Now that you have both the simulated and real robots running and can communicate with them (through `robot_sim` and `robot_real`), we can do a fun experiment. You can now write a similar loop to repeatedly show the latest status of the real robot in simulation:

In [ ]:
# ToDo: Complete this loop
import time

start_time = time.time()
while time.time()-start_time < 60:
    q = None # Replace with the joint position from the real robot
    # send the q above to the simulator
    time.sleep(0.1)

**Your Task**: Make a video of your success with this step and include in your report.

# MuJoCo XML Parsing

In [ ]:
from rob2004.rerun import RerunVisualizer
vis = visualizer = RerunVisualizer(app_name='logger', spawn=True)

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
info = ET.parse('/home/mim-server/projects/rooholla/NYU_ROB_UY_2004/rob2004/assets/robot/mujoco/pupper_v3_complete.position.fixed_base.xml')
root = info.getroot()
base = root.find('worldbody/body')
chains = base.findall('body')

In [ ]:
from scipy.spatial.transform import Rotation as R
def attrib2Pose(attrib):
    if 'pos' in attrib:
        link_pos = [float(n) for n in attrib['pos'].split(' ')] 
    else:
        link_pos = [0, 0, 0]
    if 'quat' in attrib:
        link_quat = [float(n) for n in attrib['quat'].split(' ')] 
    else:
        link_quat = [1, 0, 0 , 0]
    link_quat = link_quat[1:]+[link_quat[0]]
    link_R = R.from_quat(link_quat).as_matrix()
    T = np.eye(4)
    T[:3, :3] = link_R
    T[:3, -1] = link_pos
    return T

def rq2Pose(r, q):
    link_quat = q
    link_quat = link_quat[1:]+[link_quat[0]]
    link_R = R.from_quat(link_quat).as_matrix()
    T = np.eye(4)
    T[:3, :3] = link_R
    T[:3, -1] = r
    return T

In [ ]:
kinematic_at_rest = {}
for n in range(4):
    attrib = base.findall('body')[n].attrib
    name = attrib['name']
    B0_T_B1 = attrib2Pose(attrib)
    attrib = base.findall('body')[n].findall('body')[0].attrib
    B1_T_B2 = attrib2Pose(attrib)
    attrib = base.findall('body')[n].findall('body')[0].findall('body')[0].attrib
    B2_T_B3 = attrib2Pose(attrib)
    
    attrib = base.findall('body')[n].findall('body')[0].findall('body')[0].findall('site')[0].attrib
    B3_T_Foot = attrib2Pose(attrib)

    kinematic_at_rest[name] = dict(
        B0_T_B1=B0_T_B1,
        B1_T_B2=B1_T_B2,
        B2_T_B3 = B2_T_B3,
        B3_T_Foot = B3_T_Foot,
    )

In [ ]:
def forwardKinematics(chain):
    B0_T_B1 = chain['B0_T_B1']
    B1_T_B2 = chain['B1_T_B2']
    B2_T_B3 = chain['B2_T_B3']
    B0_T_B2 = B0_T_B1@B1_T_B2
    B0_T_B3 = B0_T_B2@B2_T_B3
    B0_T_Foot = B0_T_B3@chain['B3_T_Foot']
    return dict(
        B0_T_B1 = B0_T_B1,
        B0_T_B2 = B0_T_B2, 
        B0_T_B3 = B0_T_B3,
        B0_T_Foot = B0_T_Foot
    )    

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
T_offset = np.eye(4)
rot = R.from_euler('x', -90, degrees=True).as_matrix()
rot2 = R.from_euler('z', 90, degrees=True).as_matrix()
T_offset[:3, :3] = rot2@rot

vis.logMeshFile("/home/mim-server/projects/rooholla/NYU_ROB_UY_2004/rob2004/assets/robot/meshes/dae/Body V4 v70.001.dae",
                T_offset, 'robot/base', alpha=0)

fk = forwardKinematics(kinematic_at_rest['leg_front_r_1'])
B0_T_B1 = fk['B0_T_B1']
B1_T_mesh = rq2Pose([0, 0, -0.028], [1.32679e-06, 0, 0, 1])
B0_T_mesh = B0_T_B1@B1_T_mesh
vis.logMeshFile("/home/mim-server/projects/rooholla/NYU_ROB_UY_2004/rob2004/assets/robot/meshes/dae/Leg Assembly For Flanged v26.010.dae",
                B0_T_mesh, 'robot/leg_front_l_1')

B0_T_B2 = fk['B0_T_B2']
B2_T_mesh = rq2Pose([-0.028, 0, 0],[-2.93388e-06, 0.707108, 2.26082e-06, 0.707106])
B0_T_mesh = B0_T_B2@B2_T_mesh
vis.logMeshFile("/home/mim-server/projects/rooholla/NYU_ROB_UY_2004/rob2004/assets/robot/meshes/dae/Leg Assembly For Flanged v26.006.dae",
                B0_T_mesh, 'robot/leg_front_l_2')

B0_T_B3 = fk['B0_T_B3']
B2_T_mesh = rq2Pose([0.0685, 0.0494, -0.028],[1.32679e-06, 0, 0, 1])
B0_T_mesh = B0_T_B3@B2_T_mesh
vis.logMeshFile("/home/mim-server/projects/rooholla/NYU_ROB_UY_2004/rob2004/assets/robot/meshes/dae/Leg Assembly For Flanged v26.007.dae",
                B0_T_mesh, 'robot/leg_front_l_3')


import time
vis.logCoordinateFrame(np.eye(4), f'world/0', axis_length=0.05)

for name in kinematic_at_rest:
    if name=='leg_front_r_1':
        chain = kinematic_at_rest[name]
        result = forwardKinematics(chain)
        for k, T in enumerate(result.values()):
            time.sleep(0.5)
            vis.logCoordinateFrame(T, f'world/{name}/{k}', axis_length=0.05)
    
            
time.sleep(0.5)
vis.logCoordinateFrame(T, f'world/{name}/{k}', axis_length=0.05)